In [1]:
import math
import string
import datetime
import numpy as np
import matplotlib.pyplot
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql import Row
import pylab

%matplotlib inline

In [8]:
def config_softlayer_acct(name, auth_url, username, password):
   prefix = "fs.swift.service." + name
   hconf = sc._jsc.hadoopConfiguration()
   hconf.set(prefix + ".auth.url", auth_url)
   hconf.set(prefix + ".username", username)
   hconf.set(prefix + ".tenant", username)
   hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
   hconf.setInt(prefix + ".http.port", 8080)
   hconf.set(prefix + ".apikey", password)
   hconf.setBoolean(prefix + ".public", True)
   hconf.set(prefix + ".use.get.auth", "true")
   hconf.setBoolean(prefix + ".location-aware", False)
   hconf.set(prefix + ".password", password)

# THE ACCOUNT CONFIG HAS BEEN REMOVED- THIS PIPELINE WON'T WORK
# TEST ONLY
config_softlayer_acct("seti","https://myserver","myID","mypassword")

In [2]:
# parse thedatafile to create the full seti database

def parseLineIgnoreHeaders(line):
    words = []
    if line.startswith('UniqueId'):
        return words
    words = line
    return words

sqlContext = SQLContext(sc)

data = sc.textFile("swift://setiSignalDB.seti/*")

dataForRowNames = sc.textFile("swift://setiSignalDB.seti/xaa")
rowNames = dataForRowNames.first().split("\t")

rowNamesClear = []

#We need to modify column names with "/" in it, because of issues when call these columns later
for name in rowNames:
    if name.find("/"):
        rowNamesClear.append(name.replace("/",""))
    else: 
        rowNamesClear.append(name)

cleanData = data.map(lambda line:parseLineIgnoreHeaders(line)).filter(lambda words: len(words)>0)
rowRDD = cleanData.map(lambda line:line.split("\t")).map(lambda d:(d[0],d[1],d[2],d[3],d[4],d[5],d[6],d[7],d[8],d[9],d[10],d[11],d[12],d[13],d[14],d[15],d[16],d[17],d[18],d[19],d[20],d[21],d[22]))

fields = [StructField(field_name, StringType(), True) for field_name in rowNamesClear]
schema = StructType(fields)

exo_file = sqlContext.createDataFrame(rowRDD, schema)

Py4JJavaError: An error occurred while calling o32.partitions.
: org.apache.hadoop.fs.swift.exceptions.SwiftConfigurationException: Missing mandatory configuration option: fs.swift.service.seti.auth.url
	at org.apache.hadoop.fs.swift.http.RestClientBindings.copy(RestClientBindings.java:221)
	at org.apache.hadoop.fs.swift.http.RestClientBindings.bind(RestClientBindings.java:147)
	at org.apache.hadoop.fs.swift.http.SwiftRestClient.<init>(SwiftRestClient.java:481)
	at org.apache.hadoop.fs.swift.http.SwiftRestClient.getInstance(SwiftRestClient.java:1766)
	at org.apache.hadoop.fs.swift.snative.SwiftNativeFileSystemStore.initialize(SwiftNativeFileSystemStore.java:81)
	at org.apache.hadoop.fs.swift.snative.SwiftNativeFileSystem.initialize(SwiftNativeFileSystem.java:126)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2596)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:91)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2630)
	at org.

In [ ]:
# ... OR....
exo_file = sqlContext.read.parquet("signalDB.parquet")

In [ ]:

def deltaCalc(time):
    dt = datetime.datetime.strptime(time,"%Y-%m-%d %H:%M:%S")
            # the J2000.0 epoch start on January 1, 2000, 12:00 GMT = 4:00 PST
    delta = dt - datetime.datetime.strptime("2000-01-01 04:00:00", "%Y-%m-%d %H:%M:%S")
    return float(delta.total_seconds() / (365.25*24*60*60))

exo_rdd = exo_file.rdd.map(lambda p: Row(TgtId=p.TgtId, J2000_time=deltaCalc(p.Time),drift=float(p.DriftHzs)))

In [ ]:
doppler_fields = [StructField("J2000_time",FloatType(), True),
                  StructField("TgtId",StringType(), True),
                  StructField("drift",FloatType(), True)
                  ]

doppler_schema = StructType(doppler_fields)
print doppler_schema

In [ ]:
# Register the doppler corrected DF as a table.
calcdf = sqlContext.createDataFrame(exo_rdd, doppler_schema)
calcdf.registerTempTable("calcdb")

In [ ]:
driftSpreadDF = sqlContext.sql("SELECT TgtId, MIN(drift) minimum_d, MAX (drift) maximum_d, MIN(J2000_time) minimum_time, MAX(J2000_time) maximum_time from calcdb GROUP BY TgtId")
driftSpreadDF.count()

In [ ]:
#show results for export
driftSpreadDF.show(20000)